In [56]:
from langchain_core.messages import HumanMessage,SystemMessage,AIMessage

In [57]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [58]:
groq_api_key = os.getenv("GROQ_API_KEY")

In [59]:
from langchain_groq import ChatGroq
model = ChatGroq(model = "openai/gpt-oss-20b",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019DCCADB560>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019DCCAEDA90>, model_name='openai/gpt-oss-20b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [60]:
messages = [
    HumanMessage(content="Hi, My name is Akhil and I like Cricket the most."),
    AIMessage(content="Hello Akhil, How can I help you today?"),
    HumanMessage(content="What is my name and what is my favourite sport?")
]
model.invoke(messages)

AIMessage(content='Your name is **Akhil**, and your favourite sport is **cricket**.', additional_kwargs={'reasoning_content': 'We have a conversation: User says "Hi, My name is Akhil and I like Cricket the most." Then assistant says "Hello Akhil, How can I help you today?" Then user asks: "What is my name and what is my favourite sport?" The user is basically asking for the information that they provided earlier. The user wants the assistant to answer: name is Akhil, favourite sport is cricket. There\'s no policy conflict. It\'s not disallowed. So we can answer.'}, response_metadata={'token_usage': {'completion_tokens': 127, 'prompt_tokens': 117, 'total_tokens': 244, 'completion_time': 0.128871554, 'prompt_time': 0.006466536, 'queue_time': 0.049940314, 'total_time': 0.13533809}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_80501ff3a1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--d891621f-9031-483a-9869-

In [61]:
## Message History- It is used to wrap the messages and maintain the context of the conversation.
## In future, these messages will be loaded and sent to chain as part of the input. 
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [62]:
store ={}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Retrieve the chat history for a given session ID."""
    if(session_id not in store):
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [63]:
config = {"configurable":{"session_id":"chat1"}}

In [64]:
response = with_message_history.invoke(
    [HumanMessage(content="My name is Akhil and favourite sport is Cricket")],
    config=config
)

In [65]:
response.content

'Nice to meet you, Akhil! Cricket is such a fascinating sport—do you play it or just enjoy watching? If you have any favorite teams, players, or memorable matches, I’d love to hear about them!'

In [66]:
response2 = with_message_history.invoke(
    [HumanMessage(content="What is my name and favourite sport?")],
    config=config
)

In [67]:
response2.content

'Your name is **Akhil** and your favourite sport is **cricket**.'

In [68]:
config1 = {"configurable":{"session_id":"chat2"}}
response3 = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1
)
response3.content

'I’m not sure—I don’t have that information. Could you let me know your name?'

### Using PROMPT Templates

In [69]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer the questions to the best of your knowledge."),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [70]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [71]:
config2 = {"configurable":{"session_id":"chat3"}}
response = with_message_history.invoke(
    [
        HumanMessage(content="My name is Akhil and favourite cricketer is Rohit Sharma."),
    ],
    config = config2
)

In [72]:
response.content

'Nice to meet you, Akhil! Rohit Sharma is truly one of the most exciting batsmen to watch. What do you like most about his playing style or his most memorable innings?'

In [73]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant. Answer the questions in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [74]:
response = chain.invoke(
    {
        "messages":[HumanMessage(content="My name is akhil")],
        "language":"Telugu"
    }
)
response.content

'హాయ్ అఖిల్! మీకు ఎలా సహాయపడగలను?'

In [75]:
with_message_history = RunnableWithMessageHistory(
    chain,get_session_history,input_messages_key="messages"
)

In [76]:
config3 = {"configurable":{"session_id":"chat4"}}

In [79]:
response = with_message_history.invoke(
    {
        "messages":[HumanMessage(content="My name is Akhil and my favourite actor is Junior NTR.")],
        "language":"Telugu"
    },
    config = config3
)

In [80]:
response.content

'హలో అఖిల్! మీ పేరు అఖిల్, మీకు ఇష్టమైన నటుడు జూనియర్ NTR అని అర్థమైంది. మీకు ఇంకా ఏమైనా సహాయం చేయాలంటే చెప్పండి!'

In [81]:
with_message_history.invoke(
    {
        "messages":[HumanMessage(content="What is my Akhil and my favourite actor?")],
        "language":"Telugu"
    },
    config = config3
)

AIMessage(content='మీ పేరు అఖిల్, మీకు ఇష్టమైన నటుడు జూనియర్\u202fNTR.', additional_kwargs={'reasoning_content': 'User says: "What is my Akhil and my favourite actor?" They probably want: "Your name is Akhil and your favourite actor is Junior NTR." They want the answer in Telugu. So respond in Telugu: "మీ పేరు అఖిల్, మీ ఇష్టమైన నటుడు జూనియర్ NTR." Also maybe ask if they want more. Let\'s answer simply.'}, response_metadata={'token_usage': {'completion_tokens': 106, 'prompt_tokens': 198, 'total_tokens': 304, 'completion_time': 0.106267169, 'prompt_time': 0.00954928, 'queue_time': 0.04796432, 'total_time': 0.115816449}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_80501ff3a1', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--e11e27f6-624f-4ed3-bf02-0e4fd8a33784-0', usage_metadata={'input_tokens': 198, 'output_tokens': 106, 'total_tokens': 304})

### Managing the conversation History
If left unmanaged, the list of messages will grow continuously and overflow the context window of LLM. So this is to limit the size of messages you are passing to the llm.
'trim_messages': reduce the messages we are sending to model. Trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to keep system message always and whether to allow system_messages.

In [92]:
from langchain_core.messages import trim_messages
trimmer = trim_messages(
    max_tokens=100,
    strategy = "last",
    token_counter = model,include_system=True,
    allow_partial=False,
    start_on = "human"
)
messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="My name is Akhil"),
    AIMessage(content="Hello Akhil"),
    HumanMessage(content="I like Cricket"),
    AIMessage(content="Cricket is a great sport"),
    HumanMessage(content="What is 2+2?"),
    AIMessage(content="The answer is 4."),
    HumanMessage(content="Thanks"),
    AIMessage(content="You are welcome!")
]
trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='My name is Akhil', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello Akhil', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like Cricket', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Cricket is a great sport', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='What is 2+2?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='The answer is 4.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='You are welcome!', additional_kwargs={}, response_metadata={})]

In [93]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import Language
chain = (
    RunnablePassthrough.assign(
        messages= itemgetter("messages")|trimmer
    )|prompt|model
)
response = chain.invoke({
    "messages": messages+[HumanMessage(content="What is my name?")],
    "language":"Telugu"
})
response.content

'మీ పేరు అఖిల్.'

In [94]:
config4 = {"configurable":{"session_id":"chat5"}}
with_message_history = RunnableWithMessageHistory(
    chain,get_session_history,input_messages_key="messages"
)

In [95]:
response = with_message_history.invoke(
    {
        "messages": messages+[HumanMessage(content="What is my name?")],
        "language":"Telugu"
    },
    config = config4
)

In [96]:
response.content

'మీ పేరు అఖిల్.'